In [57]:
from sklearn.feature_extraction.text import CountVectorizer
import csv, operator 
import numpy as np
#from numpy import genfromtxt
#from numpy import loadtxt
import pandas as pd

import tensorflow as tf

# ...to serialize output arrays (bag-of-words and classes to execute supervised learning)
import pickle


### Cargamos ahora las frases prueba en una lista

Tras cargar las frases en una lista crearemos el bag-of-words asociado. 

Estas frases deben de llevar asoicadao su output para implementar un entrenamiento supervisado.

NOTAS: 
- Quizá aqui se puedan incorporar funcionalidades, nltk, para un mejor preprocesado del texto.
- Taambien podemos incluir esta celda y la siguiente en una funcion toGetBagOfWords

### ...intentamos crear el bag of words incorporando el output que necesitaremos para el entrenamiento...

In [6]:
# ...we generate the input file as csv, and we read it...

def readFromFile(name_file, encoding, delimiter):
    in_file = open(name_file, 'r', encoding=encoding)  # Abrir archivo csv
    in_data = csv.reader(in_file, delimiter = delimiter)  # Leer todos los registros
    
    in_array = np.array([])
    for in_register in in_data:
        in_array = np.append(in_array, in_register)
    in_file.close()  # Cerrar archivo
    del in_file  # Borrar objeto
    
    in_array = np.reshape(in_array, (-1,2))
    
    return in_array

#in_data = np.array(readFromFile('example-questions-supervised.csv', 'utf8', '~'))
#print(in_data)

### esta lista de vectores debe de ser la entrada de la red neuronal...

In [7]:
def createBagOfWords(corpus):
    #print(corpus)
    
    vectorizer = CountVectorizer()
    #print( vectorizer.fit_transform(corpus).todense() )
    #print( vectorizer.vocabulary_ )

    bag_of_words = vectorizer.fit_transform(corpus).todense()
    #print(bag_of_words)
    vocabulary = vectorizer.vocabulary_
    
    return vocabulary, bag_of_words
    
#createBagOfWords(in_data[:,0])

In [41]:
def vectorizeClassesOfInput(supervised_output): 
    print(supervised_output)
    print(type(supervised_output))
    
    ## ...we transform output variable (categories for supervised learning) in binary-vector shape...
    # ...firstly we trim the elements...
    supervised_output = np.char.strip(supervised_output)
    
    # ...and we apply lowercase to elements...
    supervised_output = np.char.lower(supervised_output)
    #print(supervised_output)
    
    # ...and we get the unique elements...
    unique_supervised_output = np.unique(supervised_output)
    #print(unique_supervised_output)
   
    # ...we create a zeros dataframe with dimension ( length(supervised_output)xlength(unique_supervised_output) )
    binary_supervised_output = pd.DataFrame(0, index=np.arange(len(supervised_output)), columns=unique_supervised_output)
    #print(binary_supervised_output)

    for i in range(len(supervised_output)):
        binary_supervised_output.loc[i, supervised_output[i]] = 1
    
    
    #print(binary_supervised_output)
    
    # ...we convert dataFrame-pandas to numpy-array
    binary_supervised_output = binary_supervised_output.values
    #print(binary_supervised_output)
    
    return binary_supervised_output
    
#test = np.array(['personas', 'recursos', 'recursos', 'Organizaciones', 'personas', 'recursos '])
#vectorizeClassesOfInput(in_data[:,1])
#vectorizeClassesOfInput(test)

In [64]:
def fromSentencesToBagOfWords(name_file, encoding, delimiter):
    
    # read data
    in_data = np.array(readFromFile(name_file, encoding, delimiter))
    print(in_data)
    
    # create bag of words with first column in "in_data"
    dic, bow = createBagOfWords(in_data[:,0])
    print(dic)
    print(bow)
    
    bow_class = vectorizeClassesOfInput(in_data[:,1]) 
    
    print(bow_class)
    
    # ...to share variables...
    #%store bow
    #del bow
    #%store bow_class
    #del bow_class
    
    # ...to read store variables...
    #%store -r bow_class
    
    # ...serializing arrays with "pickle" package...
    #a = # some NumPy array
    #bow_serialized = pickle.dump(bow, protocol=0) # protocol 0 is printable ASCII
    #bow_class_serialized = pickle.dump(bow_class, protocol=0)
    #deserialized_bow = pickle.loads(bow_serialized)
    
    with open('bows.pkl', 'wb') as objs_in:  # Python 3: open(..., 'wb')
        pickle.dump([bow, bow_class], objs_in)
    
    #print(bow_serialized)
    
fromSentencesToBagOfWords('example-questions-supervised.csv', 'utf8', '~')

[['dime autores que saben de inteligencia artificial' 'personas']
 ['¿sabes articulos que traten de ontologias?' 'recursos']
 ['libros sobre computación cuántica' 'recursos']
 ['¿en qué universidades se estudia web semantica?' 'organizaciones']
 ['centro de investigacion en web semantica' 'organizaciones']
 ['noticias de neurociencia' 'recursos ']]
{'autores': 2, 'articulos': 0, 'dime': 7, 'cuántica': 5, 'neurociencia': 13, 'de': 6, 'estudia': 9, 'web': 25, 'traten': 23, 'que': 16, 'semantica': 21, 'computación': 4, 'artificial': 1, 'en': 8, 'investigacion': 11, 'sobre': 22, 'noticias': 14, 'se': 20, 'qué': 17, 'saben': 18, 'centro': 3, 'inteligencia': 10, 'libros': 12, 'universidades': 24, 'sabes': 19, 'ontologias': 15}
[[0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1]
 [0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0